# **Ejercicios de pair programming Módulo 3 Sprint 1**
## **Regresión Logística: Lección 2 - Procesado**
---


Usando el mismo dataset que usatéis ayer, los objetivos de los ejercicios de hoy son:
- Estandarizar las variables numéricas de vuestro set de datos
  
- Codificar las variables categóricas. Recordad que tendréis que tener en cuenta si vuestras variables tienen orden o no.
  
- Chequear si vuestros datos están balanceados. En caso de que no lo estén utilizad algunas de las herramientas aprendidas en la lección para balancearlos.
  
- Guardad el dataframe con los cambios que habéis aplicado para utilizarlo en la siguiente lección.

In [2]:
# Tratamiento de los datos
# ========================
import numpy as np
import pandas as pd

# Librerías para la visualización de los datos
# ============================================
import matplotlib.pyplot as plt
import seaborn as sns

# Estandarización variables numéricas y Codificación variables categóricas
# ========================================================================
from sklearn.preprocessing import RobustScaler

# Gestión datos desbalanceados
# ============================
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek

# Separación de los datos en train y test
# =======================================
from sklearn.model_selection import train_test_split

# Configuración de warnings
# =========================
import warnings
warnings.filterwarnings("ignore")

# Establecer preferencias de visualización
# ========================================
plt.rcParams["figure.figsize"] = (20,20)
pd.options.display.max_columns = None 


In [3]:
df = pd.read_pickle("datos/invistico_airline_eda.pkl")
df.head(5)

,satisfaction,gender,customer_type,age,type_of_travel,class,flight_distance,seat_comfort,departure_arrival_time_convenient,food_and_drink,gate_location,inflight_wifi_service,inflight_entertainment,online_support,ease_of_online_booking,onboard_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,departure_delay_in_minutes
0,satisfied,Female,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0
1,satisfied,Male,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310
2,satisfied,Female,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0
3,satisfied,Female,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0
4,satisfied,Female,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,4,3,4,2,2,0,2,4,2,5,0


In [4]:
df_numericas = df.select_dtypes(include = np.number)
df_numericas.head()

,age,flight_distance,departure_delay_in_minutes
0,65,265,0
1,47,2464,310
2,15,2138,0
3,60,623,0
4,70,354,0


### Estandarización de las VPs con Robust Scaler

Optamos por usar el método Robust Scaler porque nuetro dataframe presenta outliers 

In [5]:

scaler = RobustScaler()

scaler.fit(df_numericas)

X_escaladas = scaler.transform(df_numericas)


df_numericas_estandar = pd.DataFrame(X_escaladas, columns = df_numericas.columns)
df_numericas_estandar.head(2)

,age,flight_distance,departure_delay_in_minutes
0,1.041667,-1.400844,0.000000
1,0.291667,0.454852,25.833333


In [6]:
df.drop(["age", "flight_distance", "departure_delay_in_minutes"], axis = 1, inplace=True)
df.head()

,satisfaction,gender,customer_type,type_of_travel,class,seat_comfort,departure_arrival_time_convenient,food_and_drink,gate_location,inflight_wifi_service,inflight_entertainment,online_support,ease_of_online_booking,onboard_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding
0,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,2,2,4,2,3,3,0,3,5,3,2
1,satisfied,Male,Loyal Customer,Personal Travel,Business,0,0,0,3,0,2,2,3,4,4,4,2,3,2
2,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,3,2,0,2,2,3,3,4,4,4,2
3,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,3,3,4,3,1,1,0,1,4,1,3
4,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,3,4,3,4,2,2,0,2,4,2,5


In [7]:
df = pd.concat([df, df_numericas_estandar], axis = 1)
df.head()

,satisfaction,gender,customer_type,type_of_travel,class,seat_comfort,departure_arrival_time_convenient,food_and_drink,gate_location,inflight_wifi_service,inflight_entertainment,online_support,ease_of_online_booking,onboard_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,age,flight_distance,departure_delay_in_minutes
0,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,2,2,4,2,3,3,0,3,5,3,2,1.041667,-1.400844,0.000000
1,satisfied,Male,Loyal Customer,Personal Travel,Business,0,0,0,3,0,2,2,3,4,4,4,2,3,2,0.291667,0.454852,25.833333
2,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,3,2,0,2,2,3,3,4,4,4,2,-1.041667,0.179747,0.000000
3,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0.833333,-1.098734,0.000000
4,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,3,4,3,4,2,2,0,2,4,2,5,1.250000,-1.325738,0.000000


### Codificación con datos estandarizados ###

- Variables que NO tienen orden:

In [8]:
lista_columnas = ["gender", "customer_type", "type_of_travel"]

df_encoded = pd.DataFrame()


for columna in lista_columnas:
    df_dummies = pd.get_dummies(df[columna], prefix_sep = "_", prefix = columna, dtype = int)

    df_encoded = pd.concat([df_encoded, df_dummies], axis = 1)



In [9]:
df_encoded.head()

,gender_Female,gender_Male,customer_type_Loyal Customer,customer_type_disloyal Customer,type_of_travel_Business travel,type_of_travel_Personal Travel
0,1,0,1,0,0,1
1,0,1,1,0,0,1
2,1,0,1,0,0,1
3,1,0,1,0,0,1
4,1,0,1,0,0,1


In [10]:
df_final = pd.concat([df, df_encoded], axis = 1)
df_final.head()

,satisfaction,gender,customer_type,type_of_travel,class,seat_comfort,departure_arrival_time_convenient,food_and_drink,gate_location,inflight_wifi_service,inflight_entertainment,online_support,ease_of_online_booking,onboard_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,age,flight_distance,departure_delay_in_minutes,gender_Female,gender_Male,customer_type_Loyal Customer,customer_type_disloyal Customer,type_of_travel_Business travel,type_of_travel_Personal Travel
0,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,2,2,4,2,3,3,0,3,5,3,2,1.041667,-1.400844,0.000000,1,0,1,0,0,1
1,satisfied,Male,Loyal Customer,Personal Travel,Business,0,0,0,3,0,2,2,3,4,4,4,2,3,2,0.291667,0.454852,25.833333,0,1,1,0,0,1
2,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,3,2,0,2,2,3,3,4,4,4,2,-1.041667,0.179747,0.000000,1,0,1,0,0,1
3,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0.833333,-1.098734,0.000000,1,0,1,0,0,1
4,satisfied,Female,Loyal Customer,Personal Travel,Eco,0,0,0,3,4,3,4,2,2,0,2,4,2,5,1.250000,-1.325738,0.000000,1,0,1,0,0,1


In [11]:
df_final.drop(lista_columnas, axis = 1, inplace=True)
df_final.head(2)

,satisfaction,class,seat_comfort,departure_arrival_time_convenient,food_and_drink,gate_location,inflight_wifi_service,inflight_entertainment,online_support,ease_of_online_booking,onboard_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,age,flight_distance,departure_delay_in_minutes,gender_Female,gender_Male,customer_type_Loyal Customer,customer_type_disloyal Customer,type_of_travel_Business travel,type_of_travel_Personal Travel
0,satisfied,Eco,0,0,0,2,2,4,2,3,3,0,3,5,3,2,1.041667,-1.400844,0.000000,1,0,1,0,0,1
1,satisfied,Business,0,0,0,3,0,2,2,3,4,4,4,2,3,2,0.291667,0.454852,25.833333,0,1,1,0,0,1



- Variables que tienen orden:

In [12]:
ordinales = df.select_dtypes(include='category').drop(columns=lista_columnas)
ordinales.sample()

,class,seat_comfort,departure_arrival_time_convenient,food_and_drink,gate_location,inflight_wifi_service,inflight_entertainment,online_support,ease_of_online_booking,onboard_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding
48261,Eco,2,2,2,4,3,2,2,3,3,3,4,2,4,3


In [14]:
col_ordinales = ordinales.columns
col_ordinales

Index(['class', 'seat_comfort', 'departure_arrival_time_convenient',
       'food_and_drink', 'gate_location', 'inflight_wifi_service',
       'inflight_entertainment', 'online_support', 'ease_of_online_booking',
       'onboard_service', 'leg_room_service', 'baggage_handling',
       'checkin_service', 'cleanliness', 'online_boarding'],
      dtype='object')

In [24]:
# obtenemos las medianas de la variable respuesta por categorías
medianas = df.groupby(['class', 'satisfaction'])['gender'].count().reset_index().sort_values(by='gender')
medianas

,class,satisfaction,gender
5,Eco Plus,satisfied,4019
4,Eco Plus,dissatisfied,5392
0,Business,dissatisfied,18065
3,Eco,satisfied,22973
2,Eco,dissatisfied,35336
1,Business,satisfied,44095
